# Chapter 26: Namespaces

---

## 26.1 Understanding Namespaces

Namespaces (formerly called "internal modules") are TypeScript's original code organization mechanism. While ES Modules have become the standard for external module systems, namespaces remain valuable for organizing code within files, avoiding global scope pollution, and specific architectural patterns.

### 26.1.1 Namespaces vs ES Modules

Understanding the distinction between namespaces and ES Modules is crucial for choosing the right tool for code organization.

```typescript
// Namespace (Internal Module)
// Organizes code within a single file or across related files
namespace Validation {
  export function isEmail(value: string): boolean {
    return value.includes("@");
  }
  
  export function isPhone(value: string): boolean {
    return /^[0-9]+$/.test(value);
  }
}

// Usage within the same file
Validation.isEmail("test@example.com");

// ES Module (External Module)
// File: validation.ts
export function isEmail(value: string): boolean {
  return value.includes("@");
}

// File: consumer.ts
import { isEmail } from "./validation";
```

**Key Differences:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Namespaces vs ES Modules                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Feature           │ Namespace                 │ ES Module         │
│  ───────────────────┼───────────────────────────┼────────────────── │
│   Scope             │ File or global            │ File-based        │
│   Import/Export     │ Triple-slash reference    │ import/export     │
│   Compilation       │ Concatenated into single  │ Separate files    │
│                     │ output (or reference)     │ (usually)         │
│   Runtime           │ Single script tag or      │ Module loader     │
│                     │ bundle                    │ (Node.js, browser)│
│   Tree-shaking      │ Difficult                 │ Supported         │
│   Modern usage      │ Legacy/internal org       │ Standard practice │
│                                                                     │
│   When to use Namespaces:                                           │
│   • Organizing large files internally                               │
│   • Declaring types for non-JS values (CSS, JSON)                  │
│   • Ambient declarations                                            │
│   • Global augmentation patterns                                    │
│                                                                     │
│   When to use ES Modules:                                           │
│   • Application code                                                │
│   • Library exports                                                 │
│   • Code splitting                                                  │
│   • Modern TypeScript/JavaScript projects                           │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 26.1.2 Namespace Scoping

Namespaces create a scoping container that prevents name collisions while allowing selective exposure of members.

```typescript
// Outer namespace
namespace App {
  // Not exported - private to the namespace
  const internalConfig = {
    version: "1.0.0"
  };
  
  // Exported - accessible from outside
  export function initialize(): void {
    console.log(`Initializing v${internalConfig.version}`);
    Utils.log("App started");
  }
  
  // Nested namespace
  export namespace Utils {
    export function log(message: string): void {
      console.log(`[LOG] ${message}`);
    }
    
    export function error(message: string): void {
      console.error(`[ERROR] ${message}`);
    }
  }
  
  // Exporting alias
  export import Logger = Utils; // Create alias
}

// Accessing namespace members
App.initialize();           // ✅ OK
App.Utils.log("Hello");     // ✅ OK
App.Logger.log("Via alias"); // ✅ OK (same as Utils)
// App.internalConfig;      // ❌ Error: not exported
```

---

## 26.2 Defining Namespaces

Namespaces are defined using the `namespace` keyword followed by the namespace name and a body containing declarations.

### 26.2.1 Namespace Syntax

Basic namespace structure with various member types.

```typescript
namespace MyApplication {
  // Variables (must be exported to be accessible)
  export const VERSION = "2.0.0";
  export let debugMode = false;
  
  // Functions
  export function enableDebug(): void {
    debugMode = true;
  }
  
  export function log(message: string): void {
    if (debugMode) {
      console.log(message);
    }
  }
  
  // Classes
  export class User {
    constructor(public name: string) {}
    
    greet(): string {
      return `Hello, ${this.name}`;
    }
  }
  
  // Interfaces
  export interface Config {
    apiUrl: string;
    timeout: number;
  }
  
  // Types
  export type UserRole = "admin" | "user" | "guest";
  
  // Enums
  export enum Status {
    Active = "ACTIVE",
    Inactive = "INACTIVE"
  }
  
  // Nested namespace
  export namespace Helpers {
    export function formatDate(date: Date): string {
      return date.toISOString();
    }
  }
}

// Accessing exported members
console.log(MyApplication.VERSION);
const user = new MyApplication.User("John");
MyApplication.Helpers.formatDate(new Date());
```

### 26.2.2 Splitting Namespaces Across Files

Namespaces can be split across multiple files using the same namespace name. TypeScript merges them at compile time.

```typescript
// file: validation/string.ts
namespace Validation {
  export function isString(value: any): value is string {
    return typeof value === "string";
  }
  
  export function isEmpty(value: string): boolean {
    return value.length === 0;
  }
}

// file: validation/number.ts
namespace Validation {
  export function isNumber(value: any): value is number {
    return typeof value === "number";
  }
  
  export function isPositive(value: number): boolean {
    return value > 0;
  }
}

// file: validation/index.ts
namespace Validation {
  export function validateAll(values: any[]): boolean {
    return values.every(v => isString(v) || isNumber(v));
  }
}

// All three declarations are merged
// Validation has: isString, isEmpty, isNumber, isPositive, validateAll
```

**Compilation Considerations:**

```typescript
// To compile split namespaces, you need to:
// 1. Use --outFile to concatenate into single output
//    tsc --outFile validation.js validation/*.ts
//
// 2. Or use reference tags to maintain file order
//    /// <reference path="string.ts" />
//    /// <reference path="number.ts" />
//
// 3. Or use a bundler that understands namespace concatenation
```

---

## 26.3 Namespace Imports

When working with namespaces across files, you need to reference external namespace declarations using triple-slash reference tags or ensure proper compilation order.

### 26.3.1 Reference Tags

Triple-slash reference tags instruct the compiler about dependencies between files.

```typescript
// file: geometry/shapes.ts
namespace Geometry {
  export interface Point {
    x: number;
    y: number;
  }
  
  export class Circle {
    constructor(public center: Point, public radius: number) {}
    
    area(): number {
      return Math.PI * this.radius ** 2;
    }
  }
}

// file: geometry/utils.ts
/// <reference path="shapes.ts" />

namespace Geometry {
  export function distance(p1: Point, p2: Point): number {
    const dx = p2.x - p1.x;
    const dy = p2.y - p1.y;
    return Math.sqrt(dx * dx + dy * dy);
  }
  
  export function circumference(circle: Circle): number {
    return 2 * Math.PI * circle.radius;
  }
}

// file: main.ts
/// <reference path="geometry/shapes.ts" />
/// <reference path="geometry/utils.ts" />

const p1: Geometry.Point = { x: 0, y: 0 };
const p2: Geometry.Point = { x: 3, y: 4 };
const dist = Geometry.distance(p1, p2); // 5
```

### 26.3.2 Import Aliases

When namespaces have long names or deep nesting, import aliases provide a convenient shorthand.

```typescript
namespace Company.Application.Features.UserManagement.Services {
  export class UserService {
    getUser(id: number) {
      return { id, name: "User" };
    }
  }
}

// Without alias (verbose)
const service1 = new Company.Application.Features.UserManagement.Services.UserService();

// With import alias
import UserSvc = Company.Application.Features.UserManagement.Services.UserService;
const service2 = new UserSvc();

// Import entire namespace
import UserManagement = Company.Application.Features.UserManagement;
const service3 = new UserManagement.Services.UserService();

// Function-scoped import
function initialize() {
  import Utils = Company.Application.Utils;
  Utils.configure();
}
```

**Alias Patterns:**

```typescript
// Creating short aliases for frequently used types
namespace DataAccess {
  export interface Repository<T> {
    find(id: number): T | undefined;
    save(entity: T): void;
  }
  
  export class UserRepository implements Repository<User> {
    // implementation
  }
  
  export interface User {
    id: number;
    name: string;
  }
}

// File-level type aliases
import Repo = DataAccess.Repository;
import UserRepo = DataAccess.UserRepository;

function processRepository(repo: Repo<DataAccess.User>) {
  // Use short alias
}
```

---

## 26.4 Aliases in Namespaces

Namespace aliases simplify working with deeply nested or verbose namespace hierarchies, improving code readability and maintainability.

### 26.4.1 Creating Namespace Aliases

The `import X = Y` syntax creates aliases for namespaces, similar to variable assignment but for namespace references.

```typescript
// Deeply nested namespace structure
namespace App {
  export namespace Infrastructure {
    export namespace Data {
      export namespace Persistence {
        export class Database {
          connect(): void {
            console.log("Connected");
          }
        }
        
        export interface ConnectionOptions {
          host: string;
          port: number;
        }
      }
    }
  }
}

// Creating convenient aliases
import Persistence = App.Infrastructure.Data.Persistence;

// Now use the short alias
const db = new Persistence.Database();
const options: Persistence.ConnectionOptions = {
  host: "localhost",
  port: 5432
};

// Multiple aliases for different levels
import Data = App.Infrastructure.Data;
import PersistenceLayer = Data.Persistence;

// Using aliased types in function signatures
function setupDatabase(
  db: Persistence.Database,
  options: Persistence.ConnectionOptions
): void {
  db.connect();
}
```

### 26.4.2 Working with External Libraries

Aliases are particularly useful when working with external declaration files that use namespaces (like older jQuery or Node.js type definitions).

```typescript
// Referencing external namespace-based libraries
// jQuery example (legacy .d.ts style)
declare namespace jQuery {
  export function ajax(options: any): void;
  export function each<T>(array: T[], callback: (index: number, value: T) => void): void;
}

// Creating convenient alias
import $ = jQuery;

$.ajax({ url: "/api/data" });
$.each([1, 2, 3], (i, val) => console.log(val));

// Working with nested external types
declare namespace MyLib {
  export namespace Types {
    export interface Config {
      debug: boolean;
    }
  }
  
  export namespace Utils {
    export function setup(config: Types.Config): void;
  }
}

// Import nested types
import Config = MyLib.Types.Config;
import Utils = MyLib.Utils;

const config: Config = { debug: true };
Utils.setup(config);
```

---

## 26.5 When to Use Namespaces

While ES Modules are the modern standard for code organization, namespaces still serve specific purposes in TypeScript development.

### 26.5.1 Internal Organization

Use namespaces to organize code within large files where ES Modules would be overkill or impractical.

```typescript
// Large utility file organized with namespaces
// utils.ts

namespace StringUtils {
  export function truncate(str: string, length: number): string {
    return str.length > length ? str.slice(0, length) + "..." : str;
  }
  
  export function slugify(str: string): string {
    return str.toLowerCase().replace(/\s+/g, "-");
  }
}

namespace NumberUtils {
  export function clamp(num: number, min: number, max: number): number {
    return Math.max(min, Math.min(num, max));
  }
  
  export function round(num: number, decimals: number): number {
    const factor = 10 ** decimals;
    return Math.round(num * factor) / factor;
  }
}

namespace DateUtils {
  export function isWeekend(date: Date): boolean {
    const day = date.getDay();
    return day === 0 || day === 6;
  }
  
  export function addDays(date: Date, days: number): Date {
    const result = new Date(date);
    result.setDate(result.getDate() + days);
    return result;
  }
}

// Export the namespaces for external use
export { StringUtils, NumberUtils, DateUtils };

// Consumer can import:
// import { StringUtils } from "./utils";
// or
// import * as Utils from "./utils";
```

### 26.5.2 Non-JavaScript Value Declarations

Namespaces are ideal for declaring types associated with non-JavaScript values like CSS modules, JSON imports, or environment variables.

```typescript
// Declaring CSS Module types
// styles.d.ts
declare namespace Styles {
  export const container: string;
  export const header: string;
  export const button: string;
  export const buttonPrimary: string;
}

export = Styles;

// Usage in component
import styles from "./Component.module.css";
// TypeScript knows styles.container, styles.header, etc.

// Declaring Webpack asset imports
declare module "*.svg" {
  namespace SvgAsset {
    export const src: string;
    export const height: number;
    export const width: number;
  }
  export = SvgAsset;
}

// Declaring process.env with specific keys
declare namespace NodeJS {
  export interface ProcessEnv {
    readonly NODE_ENV: "development" | "production" | "test";
    readonly API_URL: string;
    readonly API_KEY: string;
  }
}
```

### 26.5.3 Global Augmentation

Namespaces enable adding types to global objects or extending existing global namespaces.

```typescript
// Extending the global Window object
declare global {
  namespace Window {
    interface MyAppConfig {
      version: string;
      environment: string;
    }
  }
  
  interface Window {
    __APP_CONFIG__: Window.MyAppConfig;
    customLib: {
      init: () => void;
    };
  }
}

// Usage
window.__APP_CONFIG__ = {
  version: "1.0.0",
  environment: "production"
};

// Extending Array prototype globally
declare global {
  namespace Array {
    export function fromAsync<T>(
      iterable: AsyncIterable<T>
    ): Promise<T[]>;
  }
  
  interface Array<T> {
    last(): T | undefined;
    first(): T | undefined;
  }
}

// Implementation
if (!Array.prototype.last) {
  Array.prototype.last = function<T>(this: T[]): T | undefined {
    return this[this.length - 1];
  };
}

[1, 2, 3].last(); // TypeScript knows this exists
```

### 26.5.4 Declaration Merging Patterns

Namespaces participate in declaration merging with classes, functions, and enums to create sophisticated APIs.

```typescript
// Function + Namespace merging for jQuery-style API
function $(selector: string): Element | null {
  return document.querySelector(selector);
}

namespace $ {
  export function create(tag: string): HTMLElement {
    return document.createElement(tag);
  }
  
  export function ready(callback: () => void): void {
    if (document.readyState === "loading") {
      document.addEventListener("DOMContentLoaded", callback);
    } else {
      callback();
    }
  }
  
  export const version = "1.0.0";
}

// Usage
const el = $("#app");           // Function call
const div = $.create("div");    // Static method
$.ready(() => console.log("Ready")); // Static method
console.log($.version);         // Static property

// Enum + Namespace for grouped constants with methods
enum Status {
  Pending = "PENDING",
  Active = "ACTIVE",
  Inactive = "INACTIVE"
}

namespace Status {
  export function isValid(status: string): status is Status {
    return Object.values(Status).includes(status as Status);
  }
  
  export function getLabel(status: Status): string {
    switch (status) {
      case Status.Pending: return "⏳ Pending";
      case Status.Active: return "✅ Active";
      case Status.Inactive: return "❌ Inactive";
    }
  }
}

Status.isValid("PENDING"); // true
Status.getLabel(Status.Active); // "✅ Active"
```

### 26.5.5 Migration Strategy

When migrating from namespaces to ES Modules, follow this gradual approach:

```typescript
// Step 1: Start with namespace-based code
namespace Legacy {
  export function oldFunction() {
    return "legacy";
  }
}

// Step 2: Extract to ES Module while maintaining compatibility
// legacy.ts
export function oldFunction() {
  return "legacy";
}

// Create namespace wrapper for backwards compatibility
namespace Legacy {
  export import oldFunction = require("./legacy").oldFunction;
}

// Step 3: Gradually migrate imports
// Before: namespace reference
/// <reference path="legacy.ts" />
Legacy.oldFunction();

// After: ES Module import
import { oldFunction } from "./legacy";
oldFunction();
```

---

## 26.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored namespaces, TypeScript's original module system:

**Key Takeaways:**

1. **Namespace Fundamentals**:
   - Organize code within files or across related files
   - Create scoping containers to avoid global pollution
   - Different from ES Modules (internal vs external)

2. **Defining Namespaces**:
   - Use `namespace` keyword followed by name
   - Export members to make them accessible
   - Support variables, functions, classes, interfaces, types, and nested namespaces
   - Can be split across multiple files and merged

3. **Namespace Imports**:
   - Use `/// <reference path="..." />` for file dependencies
   - Import aliases with `import Alias = Namespace.Name`
   - Shorten long namespace chains for convenience

4. **When to Use**:
   - Internal organization within large files
   - Type declarations for non-JavaScript assets (CSS, JSON)
   - Global augmentation and declaration merging
   - Legacy codebase maintenance

5. **Modern Alternatives**:
   - Prefer ES Modules for new application code
   - Use namespaces only for specific patterns
   - Migration path from namespaces to ES Modules

### Practical Exercises

**Exercise 1: Namespace Definition**

Create organized namespaces:

```typescript
// 1. Create a 'MathUtils' namespace with:
//    - Constants: PI, E
//    - Functions: add, subtract, multiply, divide
//    - Nested 'Advanced' namespace with: power, sqrt, factorial
//    - Export only what should be public

// 2. Create a 'Validation' namespace split across two file patterns:
//    - First part: isString, isNumber, isBoolean
//    - Second part: isEmail, isURL, isDate
//    - Ensure both parts merge correctly

// 3. Create a deeply nested namespace 'App.System.Core.Services':
//    - Export a Database class
//    - Create an alias 'DB' for easy access
//    - Instantiate using the alias
```

**Exercise 2: Declaration Merging**

Practice namespace merging patterns:

```typescript
// 1. Create a function 'format' that merges with a namespace:
//    - Function: format(value: number | string): string
//    - Namespace: 
//      - currency(value: number): string
//      - percent(value: number): string
//      - date(date: Date): string
//    - Demonstrate usage of both function and namespace methods

// 2. Create an enum 'LogLevel' merging with namespace:
//    - Enum: DEBUG = 0, INFO = 1, WARN = 2, ERROR = 3
//    - Namespace:
//      - getColor(level: LogLevel): string (returns hex colors)
//      - getLabel(level: LogLevel): string
//    - Use in a logger implementation

// 3. Create a class 'EventEmitter' with namespace:
//    - Class: on(event, listener), emit(event, data)
//    - Namespace:
//      - EventTypes interface
//      - create<T>() factory method
//    - Show how both static and instance methods work
```

**Exercise 3: Global Augmentation**

Extend global types:

```typescript
// 1. Extend the global 'Array' interface:
//    - Add method 'groupBy<T, K>(keyFn: (item: T) => K): Map<K, T[]>'

// 2. Extend 'Window' interface:
//    - Add '__REDUX_DEVTOOLS_EXTENSION__' property for Redux DevTools
//    - Add 'APP_CONFIG' with your app settings interface

// 3. Create a global namespace 'GlobalUtils':
//    - Available everywhere without importing
//    - Contains formatting helpers
//    - Use declare global pattern
```

**Exercise 4: Migration**

Convert namespace to ES Module:

```typescript
// Given this namespace code:
namespace OldAPI {
  export interface Config {
    endpoint: string;
  }
  
  export class Client {
    constructor(config: Config) {}
    get(url: string): Promise<any> {
      return fetch(url);
    }
  }
  
  export function createClient(config: Config): Client {
    return new Client(config);
  }
  
  export namespace Utils {
    export function buildUrl(parts: string[]): string {
      return parts.join("/");
    }
  }
}

// Convert to ES Modules:
// 1. Split into separate files (types.ts, Client.ts, utils.ts, index.ts)
// 2. Create proper exports/imports
// 3. Maintain the same public API surface
// 4. Create barrel export in index.ts
```

### Additional Resources

- **TypeScript Handbook - Namespaces**: https://www.typescriptlang.org/docs/handbook/namespaces.html
- **TypeScript Handbook - Namespaces and Modules**: https://www.typescriptlang.org/docs/handbook/namespaces-and-modules.html
- **Migrating from Namespaces**: https://www.typescriptlang.org/docs/handbook/modules.html#export--and-import--require
- **TypeScript Deep Dive - Namespaces**: https://basarat.gitbook.io/typescript/project/modules

---

## Coming Up Next: Chapter 27 - Declaration Files

In the next chapter, we will explore **Declaration Files (.d.ts)**, TypeScript's mechanism for describing the shape of JavaScript code:

- **27.1 Understanding Declaration Files** - What .d.ts files are and their purpose
- **27.2 Creating Declaration Files** - Writing declarations for variables, functions, classes, and modules
- **27.3 Declaration File Structure** - Global declarations, module declarations, UMD patterns
- **27.4 Using DefinitelyTyped (@types)** - Leveraging community type definitions
- **27.5 Writing Declaration Files for Libraries** - Best practices for library authors
- **27.6 Publishing Types with npm Packages** - Bundling and distributing type definitions

Declaration files are the bridge between TypeScript and the vast ecosystem of JavaScript libraries, enabling type safety even when working with untyped JavaScript code.